In [22]:
import requests
import pandas as pd
import pickle
import re

cats = ["serviceLevel", "mortgage", "credits", "creditCards", "debitCards", "savings", "remoteService", "other"]

In [29]:
max_dates = {}    

In [30]:
bank = 'tink'
df = pd.read_parquet(f'data/results_sravni_{bank}.parquet')
max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()

/var/folders/v9/6czrr1vx79d5qqvjt88nm8gc0000gn/T/ipykernel_89768/3270973333.py:3: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()


In [31]:
bank = 'sber'
df = pd.read_parquet(f'data/results_sravni_{bank}.parquet')
max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()

/var/folders/v9/6czrr1vx79d5qqvjt88nm8gc0000gn/T/ipykernel_89768/1164782275.py:3: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()


In [32]:
bank = 'alpha'
df = pd.read_parquet(f'data/results_sravni_{bank}.parquet')
max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()

/var/folders/v9/6czrr1vx79d5qqvjt88nm8gc0000gn/T/ipykernel_89768/1093823675.py:3: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()


In [33]:
bank = 'vtb'
df = pd.read_parquet(f'data/results_sravni_{bank}.parquet')
max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()

/var/folders/v9/6czrr1vx79d5qqvjt88nm8gc0000gn/T/ipykernel_89768/2740841581.py:3: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_dates[bank] = df.groupby('category')['time'].agg(max).to_dict()


In [34]:
with open('data/constants/max_dates.pickle', 'wb') as f:
    pickle.dump(max_dates, f)

In [8]:
with open('data/constants/max_dates.pickle', 'rb') as f:
    max_dates = pickle.load(f)

# Tinkoff

In [11]:
bank='tink'

In [12]:
from tqdm import tqdm

res = []
for cat in cats:
  res_cat = []
  f = 1
  for i in tqdm(range(1, 1001)):
    if f == 0: break
    json = requests.get(f"https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=75da44baf111f15bf7e15eb1a990450d&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={i}&pageSize=10&reviewObjectId=5bb4f769245bc22a520a6353&reviewObjectType=banks&specificProductId=&tag={cat}&withVotes=true").json()
    for item in json["items"]:
        if max_dates[bank][cat] == item['date']:
            f = 0
            break
            

    res_cat.append(( item["date"],item["text"],item["title"], item["problemSolutionDate"], item["rating"], cat))
    if len(json["items"]) == 0: break
  res.extend(res_cat)

  0%|          | 1/1000 [00:00<04:47,  3.48it/s]


In [13]:
final = pd.DataFrame(res, columns=["date", "text", "title", "solution_date", "rating", "category"])

In [14]:
reviews = final['text']

In [15]:
def process_punctuation_txt(review):
  # Обрабатываем знаки препинания
    review = re.sub(rf'(\W)', rf' \1 ', review)

    review = re.sub(rf'\s+', rf' ', review)

    for punct in '.,':
        review = re.sub(rf'([\d\{punct}]) (\{punct}) ([\d\{punct}])', rf'\1\2\3', review)

    for punct in '!?':
        review = re.sub(rf'(\{punct}) (\{punct}) (\{punct})', rf'\1\2\3', review)

    if review[-1] == ' ':
        review = review[:-1]

    if review[-1] not in '.!?':
        review += ' .'
    review += '####[]'

    return review


def add_spaces(review):
    pattern=r'(\w\w[а-яё])([А-Я]\w\w)'
    review=re.sub(pattern, r'\1 \2', review)
    return review



def process_reviews(reviews):

    reviews = reviews.apply(lambda x: re.sub(r'\s+', ' ', x).strip())
    reviews = reviews.apply(add_spaces)
    reviews = reviews.apply(process_punctuation_txt)
    reviews = reviews.apply(lambda x: x.replace('& quot ;', '"'))

    return reviews.values.tolist()

In [16]:
reviews = process_reviews(reviews)

In [17]:
current = pd.read_parquet(f'data/results_sravni_{bank}.parquet')
current

,text,pred,pred_text,time,rating,old_text,category
0,Оформил карточку Тинькофф еще лет 8 - 10 лет н...,"[([362], [365, 366], 'NEG')]","[('сервис', 'не работает', 'NEG')]",2024-03-18T06:57:04.493153Z,1,Оформил карточку Тинькофф еще лет 8-10 лет наз...,serviceLevel
1,По ошибке банка также сменили мне тариф в Тинь...,"[([106], [117], 'NEG')]","[('представитель', 'обманул', 'NEG')]",2024-03-13T15:11:28.958801Z,1,По ошибке банка также сменили мне тариф в Тинь...,serviceLevel
2,Навязали кредитную карту в аэропорту кредитную...,"[([2], [0], 'NEG')]","[('карту', 'Навязали', 'NEG')]",2024-03-10T04:21:18.122176Z,1,Навязали кредитную карту в аэропорту кредитную...,serviceLevel
3,К сожалению приходится прощаться с Тинькоф бан...,[],[],2024-02-14T15:55:48.714485Z,1,К сожалению приходится прощаться с Тинькоф бан...,serviceLevel
4,I use the loyalty system from Tinkoff in every...,[],[],2024-02-09T22:09:08.406309Z,1,I use the loyalty system from Tinkoff in every...,serviceLevel
...,...,...,...,...,...,...,...
19862,"Открывала ИП , обратилась в Тинькофбанк , так ...",[],[],2021-07-31T08:43:14.60976Z,1,"Открывала ИП, обратилась в Тинькофбанк, так ка...",other
19863,"Я не являюсь клиентом банка , но пришлось с ни...","[([133], [134, 135, 136], 'NEG'), ([153], [154...","[('ситуация', 'никак не решается', 'NEG'), ('п...",2021-03-17T19:08:05.458614Z,1,"Я не являюсь клиентом банка, но пришлось с ни ...",other
19864,10.03.20 купила билеты в Вильнюс на 20 марта ....,[],[],2020-03-27T13:33:28.583Z,3,10.03.20 купила билеты в Вильнюс на 20 марта. ...,other
19865,"Сами банковские услуги удобные , и оформление ...","[([99], [98], 'POS'), ([99], [95], 'POS'), ([8...","[('отношение', 'лояльное', 'POS'), ('отношение...",2019-12-10T07:24:33.41Z,4,"Сами банковские услуги удобные, и оформление б...",other


In [ ]:
current

In [ ]:
final.to_parquet("tink.parquet", index=False)

# VTB

```https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=75da44baf111f15bf7e15eb1a990450d&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={i}&pageSize=10&reviewObjectId=5bb4f767245bc22a520a6081&reviewObjectType=banks&specificProductId=&withVotes=true```

In [ ]:
from tqdm import tqdm

cats = ["serviceLevel", "mortgage", "credits", "creditCards", "debitCards", "savings", "remoteService", "other"]
res_vtb = []
for cat in cats:
  res_cat = []
  for i in tqdm(range(1, 1001)):
    json = requests.get(f"https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=75da44baf111f15bf7e15eb1a990450d&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={i}&pageSize=10&reviewObjectId=5bb4f767245bc22a520a6081&reviewObjectType=banks&specificProductId=&tag={cat}&withVotes=true").json()
    for item in json["items"]:
      res_cat.append(( item["date"],item["text"],item["title"], item["problemSolutionDate"], item["rating"], cat))
    if len(json["items"]) == 0: break
  res_vtb.extend(res_cat)

  0%|          | 4/1000 [00:02<09:41,  1.71it/s]


In [ ]:
final = pd.DataFrame(res_vtb, columns=["date", "text", "title", "solution_date", "rating", "category"])

In [ ]:
len(final)

2579

In [ ]:
final.to_parquet("vtb.parquet", index=False)

# Alpha

``https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=75da44baf111f15bf7e15eb1a990450d&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex=1&pageSize=10&reviewObjectId=5bb4f767245bc22a520a60fb&reviewObjectType=banks&specificProductId=&withVotes=true``

In [ ]:
from tqdm import tqdm

cats = ["serviceLevel", "mortgage", "credits", "creditCards", "debitCards", "savings", "remoteService", "other"]
res_alpha = []
for cat in cats:
  res_cat = []
  for i in tqdm(range(1, 1001)):
    json = requests.get(f"https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=75da44baf111f15bf7e15eb1a990450d&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={i}&pageSize=10&reviewObjectId=5bb4f767245bc22a520a60fb&reviewObjectType=banks&tag={cat}&specificProductId=&withVotes=true").json()
    for item in json["items"]:
      res_cat.append(( item["date"],item["text"],item["title"], item["problemSolutionDate"], item["rating"], cat))
    if len(json["items"]) == 0: break
  res_alpha.extend(res_cat)

  3%|▎         | 29/1000 [00:16<09:23,  1.72it/s]


In [ ]:
final = pd.DataFrame(res_alpha, columns=["date", "text", "title", "solution_date", "rating", "category"])

In [ ]:
final.iloc[0]

date                                   2024-04-19T07:48:18.476662Z
text             Добрый день! Хочу выразить огромную благодарно...
title                                               Благодарность.
solution_date                                                 None
rating                                                           5
category                                              serviceLevel
Name: 0, dtype: object

In [ ]:
final.to_parquet("alpha.parquet", index=False)

In [ ]:
!python --version

Python 3.10.12


# Sber

In [ ]:
from tqdm import tqdm

cats = ["serviceLevel", "mortgage", "credits", "creditCards", "debitCards", "savings", "remoteService", "other"]
res_sber = []
for cat in cats:
  res_cat = []
  for i in tqdm(range(1, 1001)):
    json = requests.get(f"https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=75da44baf111f15bf7e15eb1a990450d&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={i}&pageSize=10&reviewObjectId=5bb4f767245bc22a520a609b&reviewObjectType=banks&specificProductId=&tag={cat}&withVotes=true").json()
    for item in json["items"]:
      res_cat.append(( item["date"],item["text"],item["title"], item["problemSolutionDate"], item["rating"], cat))
    if len(json["items"]) == 0: break
  res_sber.extend(res_cat)

  2%|▏         | 15/1000 [00:07<08:15,  1.99it/s]


In [ ]:
len(res_alpha)

3627

In [ ]:
final = pd.DataFrame(res_sber, columns=["date", "text", "title", "solution_date", "rating", "category"])

In [ ]:
len(final)

3627

In [ ]:
final.to_parquet("sber_cats.parquet", index=False)

In [ ]:
df = pd.read_csv("/content/test_selling_pandas.csv")